In [24]:
from prelu_cnn import CNN

from datasets import load_from_disk

# Load training dataset
dataset_path = "../processed_datasets/imagenet_processor"
ds = load_from_disk(dataset_path)
ds = ds['train']




In [25]:
ds['pixel_values'].shape

torch.Size([1000, 3, 224, 224])

In [26]:
inputs = ds['pixel_values']
labels = ds['labels']

In [28]:
import torch
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model and inputs to GPU if available
model = CNN(use_prelu=False, use_builtin_conv=True).to(device)

# Move tensors to the same device as the model
inputs = inputs.to(device)
labels = labels.to(device)

outputs = model(inputs)
outputs

Using device: cuda


tensor([[-0.0947, -0.0165,  0.0874,  ..., -0.1580, -0.0219, -0.2634],
        [ 0.1340,  0.1958,  0.2236,  ..., -0.3096,  0.0041, -0.0962],
        [-0.0111, -0.2845,  0.1285,  ..., -0.2256,  0.1135, -0.1852],
        ...,
        [-0.4496, -0.3307,  0.1620,  ..., -0.1684,  0.2932, -0.5237],
        [-0.1339,  0.2442, -0.1870,  ..., -0.4475,  0.1409, -0.2245],
        [ 0.2570,  0.3836,  0.0375,  ..., -0.0107,  0.1787, -0.0538]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [29]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions = np.argmax(outputs.detach().cpu().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.cpu().numpy())
print("Correct predictions:", (predictions == labels.cpu().numpy()).sum(), "out of", len(predictions))


Predictions: [455 837 432 354 640 554 867 259 464 929 752  32 121 152 694 152 442 317
 317 183 664 102 656 477   9 731 121 431 714 837 725 837 446 442   9 431
 694 853 583 534 899 294 229 837 275 317 752 497 941 250 779 478 242 394
 771 114 446 958  50 313 822 450 358 314 540 846 744 480   9 183   9 530
 384   9  32   9 317 313   9 510 317 878 480 889  42 313  80 867 609 672
 480 656 182  32 478 121 664 664 779 664 116 478   9 664 976 846 384 431
   2 247 622  32 837 143 878 741 314 741 767 114 317 975 699 352  32  42
   9   9 250 817 431 837 804  23 865 183 152 725 976 744  42 528 362 484
  18  32 837  23 102 446 110  78 368   9 867   9 450 744   9 440 959 846
 431 589 364 744 317 845 653 744 102 838  73 294 200 798 547 418  52 464
  43 622 313   9 583 959 687 313 314 440   9 998 867 313 431 317 317  22
 622 317 560  78 478 837 694 125 469 837 442 250 846   9 501 387 878  32
 362 765 859 431 789 554  42 478 425 719 310 317 810 850   9 431  78 317
   9 656 612 556 294 725  52 294 327 2

In [30]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(6.9320, device='cuda:0', grad_fn=<NllLossBackward0>)